In [1]:
import praw
import json

In [2]:
with open("config.json", "r") as jsonfile:
    data = json.load(jsonfile)  # Reading the config file
#     print("Config data read successful", data)

In [3]:
from collections import namedtuple  # Each element will be assigned a field
import sys
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession,SQLContext
# Allow us to order things in decscending order
from pyspark.sql.functions import desc

# create spark configuration
conf = SparkConf()
conf.setAppName("RedditStreamApp")
# create spark context with the above configuration
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
# create the Streaming Context from the above spark context with interval size 10 seconds
ssc = StreamingContext(sc, 10)
# setting a checkpoint to allow RDD recovery
####ssc.checkpoint("checkpoint_RedditApp")
# read data from port 5590
# socket_stream = ssc.socketTextStream("localhost", 5590)


# sc = SparkContext() # Establish an entry point of Apache Spark functionality.
# ssc = StreamingContext(sc, 10) # Create streaming context with intervals of 10 seconds.
sqlContext = SQLContext(sc) # Initialising Spark SQL functionalities.

socket_stream = ssc.socketTextStream("127.0.0.1", 5590)


C:\Users\marke\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyspark\streaming\context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(
C:\Users\marke\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyspark\sql\context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
### Modify the code here to match your logic in reddit_producer.py
#**********************
lines = socket_stream.window(20)
fields = ("author",'date','time')
RedditComment = namedtuple('RedditComment', fields)

In [5]:
def get_sql_context_instance(spark_context):
    if ('sqlContextSingletonInstance' not in globals()):
        globals()['sqlContextSingletonInstance'] = SQLContext(spark_context)
    return globals()['sqlContextSingletonInstance']


def test_content(string):
    print(string)
    return eval(string)
    
def get_avg_word(df):

    return 0

def process_rdd(time, rdd):
    print("----------- %s -----------" % str(time))
    try:
        # Get spark sql singleton context from the current context
        sql_context = get_sql_context_instance(rdd.context)

        
        # modify the mapping to account for data dictionary now being passed to it
        row_rdd = rdd.map(lambda rec: RedditComment(rec['author'],rec['date'],rec['time']))
        comments_df = sql_context.createDataFrame(row_rdd)
        comments_df.registerTempTable("comments")
        comments_df_sql = sql_context.sql(
            "select * from comments limit 10")
        comments_df_sql.show()

        #####
        rows = comments_df.collect()
        name = rows['author']


    except:
        e = sys.exc_info()[0]
        print("Error: %s" % e)



(lines.map(lambda text: test_content(text)).foreachRDD(process_rdd))

In [6]:
ssc.start()
# ssc.awaitTermination()  

----------- 2023-12-03 17:58:20 -----------
Error: <class 'ValueError'>
----------- 2023-12-03 17:58:30 -----------


C:\Users\marke\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyspark\sql\dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


+--------------------+----------+--------+
|              author|      date|    time|
+--------------------+----------+--------+
|    spookyhellkitten|2023-12-03|17:45:47|
|       After_Ad_8841|2023-12-03|17:45:48|
|      Lemon_head_guy|2023-12-03|17:45:49|
| Spiritual-Wind-3898|2023-12-03|17:45:50|
|       ThorneInMyEye|2023-12-03|17:45:54|
|CalligrapherShort121|2023-12-03|17:46:04|
|        Pitsmithy_89|2023-12-03|17:46:14|
|          jmochicago|2023-12-03|17:46:27|
|      itsshakespeare|2023-12-03|17:46:27|
|  Mantequilla_Stotch|2023-12-03|17:46:29|
+--------------------+----------+--------+

Error: <class 'TypeError'>
----------- 2023-12-03 17:58:40 -----------
+--------------------+----------+--------+
|              author|      date|    time|
+--------------------+----------+--------+
|    spookyhellkitten|2023-12-03|17:45:47|
|       After_Ad_8841|2023-12-03|17:45:48|
|      Lemon_head_guy|2023-12-03|17:45:49|
| Spiritual-Wind-3898|2023-12-03|17:45:50|
|       ThorneInMyEye|202

In [7]:
ssc.stop()

Error: <class 'py4j.protocol.Py4JJavaError'>
